### Part 0: Initialisation and saving qHome

In [62]:
import robotic as ry
import time

In [63]:
C = ry.Config()
C.addFile(ry.raiPath('Question-01/environment/cargo.g'))
C.addFile(ry.raiPath('Question-01/environment/cargobot.g'))
C.addFile(ry.raiPath('Question-01/environment/maze.g'))

C.view()
qHome = C.getJointState()
print(qHome)

[ 0.   0.   0.   0.  -0.5  0.  -2.   0.   2.  -0.5]


In [64]:
# del C
# C.view()
# C.getFrameNames()
# qHome = C.getJointState()
# print(qHome)
# print(C.getFrameNames())

### 1.1 Part A: gripper to cargo handle position KOMO_1

In [65]:
# komo = ry.KOMO(C, phases=1, slicesPerPhase=30, kOrder=1, enableCollisions=False)
komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 20, 5., 0)

# add collision constraints
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

# move the gripper to the cargo position
komo.addObjective(
    times=[],
    feature=ry.FS.positionDiff,
    frames=["l_gripper", "cargo_handle"],
    type=ry.OT.eq,
    scale=[1e1]
)

# komo.addObjective(
#     times=[],
#     feature=ry.FS.positionDiff,
#     frames=["l_gripper", "cargo_handle"],
#     type=ry.OT.sos,
#     scale=[1e1]
# )

# align the grippper with the cargo_handle
komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', 'cargo_handle'], ry.OT.eq, [1e1])
# komo.addObjective([], ry.FS.scalarProductXY, ['l_panda_base', 'cargo_handle'], ry.OT.eq, [1e1])



ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
q_gripper_to_cargo = komo.getPath()

# for t in range(q_gripper_to_cargo.shape[0]):
#     C.setJointState(q_gripper_to_cargo[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

# C.view()
if ret.feasible:
    print("KOMO_1 is feasible")
else:
    print("KOMO_1 is NOT feasible")

KOMO_1 is feasible


### 1.1 Part A: l_panda_base to goal position KOMO_2

In [66]:
komo = ry.KOMO(C, phases=1, slicesPerPhase=20, kOrder=1, enableCollisions=False)
# komo = ry.KOMO()
# komo.setConfig(C, True)
# komo.setTiming(1., 20, 5., kOrder=1)

# add control objective to stay close to the initial joint configuration
komo.addControlObjective([], 0, 1e-1)

# add collision constraints
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

# Add objective to move cargo to goal area
komo.addObjective(
    times=[0., 1.],
    feature=ry.FS.positionDiff,
    frames=["l_panda_base", "goal_area"],
    type=ry.OT.sos
)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
q_cargobot_to_goal_area = komo.getPath()



# for t in range(q_cargobot_to_goal_area.shape[0]):
#     C.setJointState(q_cargobot_to_goal_area[t])
#     C.view(False, f'waypoint {t}')
#     time.sleep(0.1)

if ret.feasible:
    print("KOMO_2 is feasible")
else:
    print("KOMO_2 is NOT feasible")

KOMO_2 is feasible


### 1.1 Part A: Visualisation of KOMO_1 and KOMO_2

In [67]:
for t in range(q_gripper_to_cargo.shape[0]):
    C.setJointState(q_gripper_to_cargo[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)
    
for t in range(q_cargobot_to_goal_area.shape[0]):
    C.setJointState(q_cargobot_to_goal_area[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

# C.setJointState(qHome)
# C.view()
# komo.view_play(False, 2)

### 1.2 Part B: RRT pathfinding (KOMO_1)

In [68]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.5, 'rrt/verbose': 0}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

C.setJointState(qHome)


while (not foundRRTPath and numOfTrials < 15):
  rrt = ry.PathFinder()
  rrt.setProblem(C, [qHome], [q_gripper_to_cargo[0]])
  ret = rrt.solve()
  path = ret.x
  foundRRTPath = ret.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  
print()
print()
print()
print()
print()
print()
print()

if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")








*** Found path at trial num # 1  ***


In [69]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

### 1.2 Part B: RRT pathfinding (KOMO_2)

In [ ]:
ry.params_clear()
ry.params_add({'rrt/stepsize':.1, 'rrt/verbose': 4}) #verbose=3 makes it very slow, and displays result, and verbose=4 waits keypress..

foundRRTPath = False
numOfTrials = 0

rrt = ry.PathFinder()

while (not foundRRTPath and numOfTrials < 10):
  rrt.setProblem(C, [q_gripper_to_cargo[0]], [q_cargobot_to_goal_area[0]])
  ret = rrt.solve()
  path = ret.x
  foundRRTPath = ret.feasible
  numOfTrials = numOfTrials + 1
  # print("*** Trial num #", numOfTrials, " result: ", foundRRTPath)
  

if foundRRTPath:
  print("*** Found path at trial num #", numOfTrials, " ***")
else:
  print("** No path found ***")

RRT queries=403 tree sizes = 9 60
RRT queries=696 tree sizes = 23 77
RRT queries=989 tree sizes = 33 98
RRT queries=1258 tree sizes = 38 116
RRT queries=1539 tree sizes = 51 130
RRT queries=1844 tree sizes = 67 149
RRT queries=2149 tree sizes = 87 164
RRT queries=2412 tree sizes = 97 175
RRT queries=2723 tree sizes = 119 190
RRT queries=3034 tree sizes = 145 201
RRT queries=3399 tree sizes = 180 221
RRT queries=3809 tree sizes = 231 240
  -- rrt success: queries:3885 tree sizes: 243 245
  path-length=228
*** Found path at trial num # 1  ***


In [43]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [ ]:
# C.attach("l_gripper", "cargo_handle")
# del komo
# C.getFrame("base_origin").setPosition([1.7, -1.6, 0.1])
# C.getFrame("base_origin").setQuaternion([1, 0, 0, 0])
# C.view()